# 챕터 20: 속성 디스크립터

## 20.1 디스크립터 예: 속성 검증

* 디스크립터 클래스: 디스크립터 프로토콜을 구현하는 클래스.
* 관리 대상 클래스: 디스크립터 객체를 클래스 속성으로 선언하는 클래스.
* 디스크립터 객체: 관리 대상 클래스 속성으로 선언된, 디스크립터 클래스의 객체.
* 관리 대상 객체: 관리 대상 클래스의 객체.
* 저장소 속성: 관리 대상 객체 안의 관리 대상 속성값을 담을 속성.
* 관리 대상 속성: 디스크립터 객체에 의해 관리되는 대상 클래스 안의 공개 속성으로, 이 속성의 값은 저장소 속성에 저장된다. 즉, 디스크립터 객체와 저장소 속성이 관리 대상 속성에 대한 기반을 제공한다.

In [ ]:
# 예제 20-1: bulkfood_v3.py: LineItem의 속성을 관리하는 Quantity 디스크립터
class Quantity:
    
    def __init__(self, storage_name):
        self.storage_name = storage_name
        
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
# LineItem 버전 #4: 자동 저장소 속성명
# 예제 20-2 bulkfood_v4.py: 각 Quantity 디스크립터는 고유한 storage_name을 가진다.
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.formate(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
# 예제 20-3: bulkfood_v4b.py: 관리되는 클래스를 통해 호출되면 get()은 디스크립터 자체에 대한 참조를 반환한다.
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.formate(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

In [1]:
# LineItem 버전 #5:새로운 디스크립터형
# model_v5.py: 리팩토링한 디스크립터 클래스

import abc 

class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.formate(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """검증된 값을 반환하거나 ValueError를 발생시킨다."""

class Quantity(Validated):
    """0보다 큰 수"""
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    """최소 하나 이상의 비공백 문자가 들어 있는 문자열"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

SyntaxError: unexpected EOF while parsing (<ipython-input-1-776505885c78>, line 51)

In [ ]:
# 예제 20-7: bulkfood_v5.py: Quantity와 NonBlank 디스크립터를 사용한 LineItem
import model_v5 as model

class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price